**Instructions:**

- For questions that require coding, you need to write the relevant code and display its output. Your output should either be the direct answer to the question or clearly display the answer in it.
- For questions that require a written answer (sometimes along with the code), you need to put your answer in a Markdown cell. Writing the answer as a comment or as a print line is not acceptable.
- You need to render this file as HTML using Quarto and submit the HTML file. **Please note that this is a requirement and not optional.** A submission cannot be graded until it is properly rendered.

Import all the libraries and tools you need below.

In [2]:
# Import all libraries/packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, hamming_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, ShuffleSplit
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR
# import random forest model
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
# Imoport the feature for multi-label regression
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor, ClassifierChain
from sklearn.datasets import make_multilabel_classification


### 1)

Read the data from **X_train.csv**, **y_train.csv**, **X_test.csv**, and **y_test.csv**. The predictors (X's) are rhythmic and timbre features extracted from a number of songs. The responses (y's) are the emotion class labels for each song. **(2.5 points)**

In [3]:
# Read data from all dataset
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

# Check data
X_train.head()


,"('Mean_Acc1298_Mean_Mem40_Centroid', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_Rolloff', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_Flux', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_MFCC_0', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_MFCC_1', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_MFCC_2', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_MFCC_3', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_MFCC_4', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_MFCC_5', 'NUMERIC')","('Mean_Acc1298_Mean_Mem40_MFCC_6', 'NUMERIC')",...,"('Std_Acc1298_Std_Mem40_MFCC_11', 'NUMERIC')","('Std_Acc1298_Std_Mem40_MFCC_12', 'NUMERIC')","('BH_LowPeakAmp', 'NUMERIC')","('BH_LowPeakBPM', 'NUMERIC')","('BH_HighPeakAmp', 'NUMERIC')","('BH_HighPeakBPM', 'NUMERIC')","('BH_HighLowRatio', 'NUMERIC')","('BHSUM1', 'NUMERIC')","('BHSUM2', 'NUMERIC')","('BHSUM3', 'NUMERIC')"
0,0.034741,0.089665,0.091225,-73.302422,6.215179,0.615074,2.037160,0.804065,1.301409,0.558576,...,0.118630,0.094923,0.051035,68.0,0.014937,136.0,2.0,0.245457,0.105065,0.405399
1,0.081374,0.272747,0.085733,-62.584437,3.183163,-0.218145,0.163038,0.620251,0.458514,0.041426,...,0.070075,0.041565,0.295031,70.0,0.276366,140.0,2.0,0.343547,0.276366,0.710924
2,0.110545,0.273567,0.084410,-65.235325,2.794964,0.639047,1.281297,0.757896,0.489412,0.627636,...,0.079917,0.085821,0.161574,61.0,0.000000,183.0,3.0,0.188693,0.045941,0.457372
3,0.042481,0.199281,0.093447,-80.305152,5.824409,0.648848,1.754870,1.495532,0.739909,0.809644,...,0.129145,0.122330,0.043012,66.0,0.206562,132.0,2.0,0.102839,0.241934,0.351009
4,0.074550,0.140880,0.079789,-93.697749,5.543229,1.064262,0.899152,0.890336,0.702328,0.490685,...,0.284196,0.189988,0.029308,100.0,0.144039,200.0,2.0,0.195196,0.310801,0.683817


### 2)

Print the first five rows of either `y_train` or `y_test`. You should observe that each observation has multiple class labels and it is possible for an observation to have multiple Class 1 values.

**(2.5 points)**

In [4]:
# Print first five rows of y_train or y_test
y_train.head()


,"('amazed-suprised', ['0', '1'])","('happy-pleased', ['0', '1'])","('relaxing-calm', ['0', '1'])","('quiet-still', ['0', '1'])","('sad-lonely', ['0', '1'])","('angry-aggresive', ['0', '1'])"
0,0,1,1,0,0,0
1,1,0,0,0,0,1
2,0,1,0,0,0,1
3,0,0,1,0,0,0
4,0,0,0,1,0,0


There are totally 6 responses per observation.

### 3)

Create a [Random Forest Classifier](https://scikit-learn.org/1.6/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Use 500 trees, so that its variance is reduced adequately. Leave all the other hyperparameters default; tuning their values does not change the results substantially. Use `random_state=2` for reproducibility. **(5 points)**

In [5]:
# Create a random forest (rf) classifier/ set at 500 trees 
random_forest = RandomForestClassifier(n_estimators = 500, random_state = 2)

### 4)

Train the Random Forest on the multi-label data using the **Binary Relevance** approach. You need to check the [scikit-learn documentation](https://scikit-learn.org/stable/api/sklearn.multioutput.html) for the correct object and its usage.

Evaluate the multi-label classifier on the test data, using [Hamming Loss](https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.hamming_loss.html).

**(15 points)**

In [6]:
# Binary Relevance Approach

# Change all dataframe to array
X_train = np.array(X_train)
X_test = np.array(X_test)

# Get the model workflow for classification
binary_relevance_rf = MultiOutputClassifier(random_forest).fit(X_train, y_train)

# Predict the value of y based on X_test
y_pred = binary_relevance_rf.predict(X_test)

In [7]:
# Check the cost using hamming loss
hammingloss_binary = hamming_loss(y_test, y_pred)

# Print out
hammingloss_binary

0.19636963696369636

### 5)

What does the Hamming Loss represent in terms of what the model predicts right/wrong? **(10 points)**

Hamming loss represents the fraction of labels that are incorrectly predicted across all observations. Therefore, the model with higher hamming loss predicts more wrong results while the model with lower hamming loss guarantees that it will accurately predict the classes of responses. The model is perfect if hamming loss = 0. 

In this dataset, out of $391 * 6 = 2346$ labels of responses, the model incorrectly predicts $2346 * 0.1964 \approx 461$ of them and rightly predicts $2346 - 461 = 1885$ of them.

### 6)

Train the Random Forest on the multi-label data using the **Classifier Chain** approach. Keep all the inputs (other than the base model) default. You may need to refer back to the scikit-learn documentation for the correct object.

Evaluate the multi-label classifier on the test data, using Hamming Loss. You should see the same performance as Question 4.

**(15 points)**

In [8]:
# Classifier chain method

# Create a classifier chain model
classifier_chain_rf = ClassifierChain(random_forest, random_state = 2)

# Get the y_pred output
y_pred = classifier_chain_rf.fit(X_train, y_train).predict(X_test)

In [9]:
# Find Hamming loss
hammingloss_chain = hamming_loss(y_test, y_pred)

# Print out
hammingloss_chain

0.19554455445544555

The performance of both model created through two different methods are very close to the same. Both of them have the hamming loss at approximately 0.196.

### 7)

Using the scikit-learn documentation, answer the following about the multi-label model in Question 6:

- Are you using the true or the predicted classes from the previous classifier(s) as the predictors of the next classifier in the chain?
- What is the order of class variables that you use for the chain?

**(10 points)**

Based on scikitlearn API, we apply the predicted classes from the previous classifiers as the predictors of the next classification process in the chain. This is sesible since the true classes of responses should not be known for the prediction problems.

The order of class variable I apply for my model is $None$ which basically sets the order for chain processing to the order of columns in label matrix Y.

### 8)

Repeat Question 6, only with `cv=5` as another input to the Classifier Chain. What does this change about the multi-label model?

You should see a slightly lower performance. Why is this a more realistic evaluation of the model?

**(10 points)**

In [10]:
# Classifier chain method

# Create a classifier chain model, add cv = 5
classifier_chain_rf = ClassifierChain(random_forest, cv = 5, random_state = 2)

# Get the y_pred output
y_pred = classifier_chain_rf.fit(X_train, y_train).predict(X_test)

# Find Hamming loss
hammingloss_chain_updated = hamming_loss(y_test, y_pred)

# Print out
hammingloss_chain_updated

0.1971947194719472

Hamming loss turns out to be higher because, with cross validation, the size of training dataset is bigger than the one after cross validation (With five folds, the size of training set sinks down to 80%). The more training data to work on let the classifier chain without cv performs better. 

In reality, the slightly smaller dataset of classifier chain with cv symbolizes the unseen and incomplete label dependencies during training period.

### 9)

Run the given cell below. It calculates and prints the Variation Inflation Factor (VIF) of each class variable.

VIF is a way to aggregate the multi-collinearity each variable has with all the other variables. The higher the VIF value of a variable is, the higher its total correlation is with all the other variables. **Note that having a correlation/multi-collinearity with other variables means carrying some information about other variables.**

**(0 points)**

In [11]:
# Change y_train back to dataframe
y_train = pd.DataFrame(y_train)

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
y = add_constant(y_train)
vif_data = pd.DataFrame()
vif_data["feature"] = y.columns

for i in range(len(y.columns)):
    vif_data.loc[i,'VIF'] = variance_inflation_factor(y.values, i)

print(vif_data)

                           feature        VIF
0                            const  10.591750
1  ('amazed-suprised', ['0', '1'])   1.434148
2    ('happy-pleased', ['0', '1'])   1.676727
3    ('relaxing-calm', ['0', '1'])   1.782708
4      ('quiet-still', ['0', '1'])   1.630620
5       ('sad-lonely', ['0', '1'])   1.623160
6  ('angry-aggresive', ['0', '1'])   2.126801


### 10)

Using the output of Question 9, repeat Question 8, only this time with the **most informative** order of the class variables. (Python starts counting from zero.)

You should see the best model performance in this assignment. Why is this the case?

**(15 points)**

In [13]:
# Classifier chain method, order specified

# Create a classifier chain model, add cv = 5
classifier_chain_rf = ClassifierChain(random_forest, order = [5,2,1,3,4,0], cv = 5, random_state = 2)

# Get the y_pred output
y_pred = classifier_chain_rf.fit(X_train, y_train).predict(X_test)

# Find Hamming loss
hammingloss_chain_updated = hamming_loss(y_test, y_pred)

# Print out
hammingloss_chain_updated

0.19389438943894388

This model will perform best since its first variable applied for the first prediction, which will be used along the chain, carries a significant amount of information. The more information received from predictors, the more accurate predictions can be. Thus, the most informative arrangement of class variables optimizes the available data for model construction.

### 11)

Finally, using the predictions of the best Classifier Chain (from Question 10), calculate and print the test accuracy of the model **for each emotion**. Which emotions are predicted the most and least accurately?

**(15 points)**

In [14]:
# Change y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
emotion = y_test.columns
# Get the accuracy
for i in range(6):
    print(f"Accuracy of {emotion[i]}:{accuracy_score(y_test.iloc[:,i], y_pred[i]):.4f}")

Accuracy of ('amazed-suprised', ['0', '1']):0.8119
Accuracy of ('happy-pleased', ['0', '1']):0.7822
Accuracy of ('relaxing-calm', ['0', '1']):0.7376
Accuracy of ('quiet-still', ['0', '1']):0.8663
Accuracy of ('sad-lonely', ['0', '1']):0.7921
Accuracy of ('angry-aggresive', ['0', '1']):0.8465


The emotion 'quite-still' has highest accuracy at 0.8663. On the other hand, the genre with lowest accuracy is relaxing-calm.